In [1]:
import sys
!{sys.executable} -m pip install rdflib owlrl owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.45-cp310-cp310-linux_x86_64.whl size=24077396 sha256=55329f88f7fd6b4ea273f5fbbe3daa1e1c9aa33a5d85974c9addc843446d8905
  Stored in directory: /root/.cache/pip/wheels/5c/f4/9d/249b1671d391e3feddd443c7d0eb79a732afabc9f370591271
Successfully built owlready2


In [2]:
from rdflib import Graph, RDFS, RDF, URIRef, Namespace, Literal, OWL, XSD
from rdflib.namespace import DC, FOAF
from owlready2 import *
import pandas as pd

In [3]:
ontology_file = "userKG_inferred.rdf"

In [4]:
from rdflib import Graph
userKG = Graph()
userKG.parse(ontology_file)

<Graph identifier=Nf5bd39d8c0294cd8841587001696ff68 (<class 'rdflib.graph.Graph'>)>

# **1. Patients**

### 1.1. All patients

In [80]:
def patients(graph):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient WHERE {
        ?patient a :Patient .
      }
      """

    res = graph.query(sparql)

    res = list(res)
    res = [(c1[0].n3(graph.namespace_manager)) for c1 in res]
    df = pd.DataFrame(res, columns=["Patient"])
    return df

In [81]:
patients(userKG).head()

,Patient
0,userKG:Hanna
1,userKG:John
2,userKG:Patient1
3,userKG:Patient10
4,userKG:Patient100


### 1.2. Patients who have diabetes or are at risk of developing diabetes

In [78]:
def diabetes_or_risk_diabetes_patients(graph):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patientClass ?patient WHERE {
        ?patient a :Patient .
        ?patient a ?patientClass .

        FILTER(?patientClass = :DiabetesPatient || ?patientClass = :DiabetesRiskPatient)
      }
      """

    res = graph.query(sparql)

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager)) for c1, c2 in res]
    df = pd.DataFrame(res, columns=["PatientType", "Patient"])
    return df

In [79]:
diabetes_or_risk_diabetes_patients(userKG).head()

,PatientType,Patient
0,userKG:DiabetesPatient,userKG:Hanna
1,userKG:DiabetesRiskPatient,userKG:John
2,userKG:DiabetesPatient,userKG:Patient1
3,userKG:DiabetesPatient,userKG:Patient10
4,userKG:DiabetesPatient,userKG:Patient100


# **2. Recommendations**

### 2.1. Recommendations for all patients or for a specific patient

In [86]:
def recommendations(graph, name=None):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?recommendation WHERE {
        ?patient a :Patient .

        ?patient :hasRecommendation ?recommendationInd .
        ?recommendationInd a ?recommendation .
        ?recommendation rdfs:subClassOf :Recommendation .

        FILTER(?recommendation != :Recommendation)
      }
      """

    if name != None:
        n = name.replace("userKG:", "")
        uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
        res = graph.query(sparql, initBindings={'patient': uri})
    else:
        res = graph.query(sparql)

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager)) for c1, c2 in res]
    df = pd.DataFrame(res, columns=["Patient", "Recommendation"])
    return df

In [87]:
recommendations(userKG).head()

,Patient,Recommendation
0,userKG:John,userKG:ChangeDiet
1,userKG:John,userKG:IncreaseActivity
2,userKG:John,userKG:ReduceStress
3,userKG:Patient1,userKG:ChangeDiet
4,userKG:Patient1,userKG:IncreaseActivity


In [88]:
recommendations(userKG, "userKG:John")

,Patient,Recommendation
0,userKG:John,userKG:ChangeDiet
1,userKG:John,userKG:IncreaseActivity
2,userKG:John,userKG:ReduceStress


# **3. Recommended activities**

### 3.1. Sorted values according to their priority for all patients or for a specific patient

In [89]:
def sorted_values(graph, name=None):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?value (COUNT(?secondaryValue) AS ?nSecondaryValues) WHERE {
        ?patient a :Patient .

        ?patient :hasValue ?valueInd .
        ?valueInd a ?value .
        ?value rdfs:subClassOf :Value .

        OPTIONAL {?valueInd :prioritizedOver ?secondaryValueInd .
                  ?secondaryValueInd a ?secondaryValue .
                  ?secondaryValue rdfs:subClassOf :Value}

        FILTER(?value != :Value)
        FILTER(!bound(?secondaryValue) || ?secondaryValue != :Value)
      }
      GROUP BY ?patient ?value
      ORDER BY DESC(?nSecondaryValues)
      """

    if name != None:
        n = name.replace("userKG:", "")
        uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
        res = graph.query(sparql, initBindings={'patient': uri})
    else:
        res = graph.query(sparql)

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager),
            c3.n3(graph.namespace_manager)) for c1, c2, c3 in res]
    df = pd.DataFrame(res, columns=["Patient", "Value",
                                    "NumberOfSecondaryValues"])
    return df

In [90]:
sorted_values(userKG).head()

,Patient,Value,NumberOfSecondaryValues
0,userKG:Patient12,userKG:WarmRelationships,"""11""^^xsd:integer"
1,userKG:Patient20,userKG:Security,"""11""^^xsd:integer"
2,userKG:Patient26,userKG:Family,"""11""^^xsd:integer"
3,userKG:Patient75,userKG:Family,"""11""^^xsd:integer"
4,userKG:Patient78,userKG:SelfFulfillment,"""11""^^xsd:integer"


In [91]:
sorted_values(userKG, "userKG:John")

,Patient,Value,NumberOfSecondaryValues
0,userKG:John,userKG:Security,"""3""^^xsd:integer"
1,userKG:John,userKG:Family,"""2""^^xsd:integer"
2,userKG:John,userKG:WarmRelationships,"""2""^^xsd:integer"
3,userKG:John,userKG:Work,"""1""^^xsd:integer"
4,userKG:John,userKG:Friends,"""0""^^xsd:integer"


### 3.2. Recommended activities sorted according to the values prioritization for a specific patient

In [92]:
def recommended_activities_sorted(graph, name):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?value (COUNT(?secondaryValue) AS ?nSecondaryValues)
      ?recommendedActivity WHERE {
        ?patient a :Patient .

        OPTIONAL {?patient :hasValue ?valueInd .
                  ?valueInd a ?value .
                  ?value rdfs:subClassOf :Value .}

        OPTIONAL {?valueInd :prioritizedOver ?secondaryValueInd .
                  ?secondaryValueInd a ?secondaryValue .
                  ?secondaryValue rdfs:subClassOf :Value .}

        OPTIONAL {?valueInd :relevantActivity ?recommendedActivity .}

        ?patient :hasRecommendedActivity ?recommendedActivity .
        MINUS {OPTIONAL {?patient :hasPhysicalActivityHabit ?recommendedActivity .}}

        FILTER(?value != :Value)
        FILTER(!bound(?secondaryValue) || ?secondaryValue != :Value)
      }
      GROUP BY ?patient ?value ?nSecondaryValues ?recommendedActivity
      ORDER BY DESC(?nSecondaryValues)
      """

    n = name.replace("userKG:", "")
    uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
    res = graph.query(sparql, initBindings={'patient': uri})

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager),
            c3.n3(graph.namespace_manager), c4.n3(graph.namespace_manager)) for c1, c2, c3, c4 in res]
    df = pd.DataFrame(res, columns=["Patient", "Value",
                                    "NumberOfSecondaryValues", "RcommendedActivity"])
    return df.drop_duplicates("RcommendedActivity").drop(["NumberOfSecondaryValues",
                                                          "Value"],
                                                         axis=1)

In [93]:
ra = recommended_activities_sorted(userKG, "userKG:John")
ra

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_balance
1,userKG:John,userKG:activity_bodyweight
2,userKG:John,userKG:activity_pilates
3,userKG:John,userKG:activity_stretching
4,userKG:John,userKG:activity_yoga
5,userKG:John,userKG:activity_cycling
6,userKG:John,userKG:activity_dancing
20,userKG:John,userKG:activity_running
22,userKG:John,userKG:activity_swimming
23,userKG:John,userKG:activity_weightlifting


### 3.2.1. Patient's values associated with a specific activity

In [94]:
def relevant_values(graph, name, activity):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?value ?recommendedActivity WHERE {
        ?patient a :Patient .

        ?patient :hasValue ?valueInd .
        ?valueInd a ?value .
        ?value rdfs:subClassOf :Value .

        ?patient :hasRecommendedActivity ?recommendedActivity .

        ?valueInd :relevantActivity ?recommendedActivity .

        FILTER(?value != :Value)
      }
      """

    act = activity.replace("userKG:", "")
    uri = "http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + act
    a = URIRef(uri)

    n = name.replace("userKG:", "")
    uri2 = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)

    res = graph.query(sparql, initBindings={'patient': uri2,
                                            'recommendedActivity': a})

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager),
            c3.n3(graph.namespace_manager)) for c1, c2, c3 in res]
    df = pd.DataFrame(res, columns=["Patient", "Value", "RcommendedActivity"])
    return df

In [96]:
relevant_values(userKG, "userKG:John", ra["RcommendedActivity"][2])

,Patient,Value,RcommendedActivity
0,userKG:John,userKG:Family,userKG:activity_pilates
1,userKG:John,userKG:Friends,userKG:activity_pilates
2,userKG:John,userKG:Security,userKG:activity_pilates
3,userKG:John,userKG:WarmRelationships,userKG:activity_pilates


### 3.3. Recommended activities considering challenges for all patients or for a specific patient (financial, time constraints and injuries)

In [97]:
def recommended_activities_challenges(graph, name=None):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?recommendedActivity WHERE {
        ?patient a :Patient .

        ?patient :hasRecommendedActivity ?recommendedActivity .
        MINUS {OPTIONAL {?patient :hasPhysicalActivityHabit ?recommendedActivity .}}

        MINUS {OPTIONAL {?patient :hasChallenge ?challenge .
                         ?recommendedActivity :incompatibleWithConstraint ?challenge .}}

        MINUS {OPTIONAL {?patient :hasInjury ?injury .
                         ?recommendedActivity :incompatibleWithInjury ?injury .}}
      }
      """

    if name != None:
        n = name.replace("userKG:", "")
        uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
        res = graph.query(sparql, initBindings={'patient': uri})
    else:
        res = graph.query(sparql)

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager)) for c1, c2 in res]
    df = pd.DataFrame(res, columns=["Patient", "RcommendedActivity"])
    return df

In [98]:
recommended_activities_challenges(userKG).head()

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_pilates
1,userKG:John,userKG:activity_yoga
2,userKG:Patient1,userKG:activity_pilates
3,userKG:Patient1,userKG:activity_yoga
4,userKG:Patient10,userKG:activity_pilates


In [100]:
recommended_activities_challenges(userKG, "userKG:John")

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_pilates
1,userKG:John,userKG:activity_yoga


### 3.4. Recommended activities considering weather for all patients or for a specific patient

In [101]:
def recommended_activities_weather(graph, date="2024-03-21", name=None):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT DISTINCT ?patient ?recommendedActivity WHERE {
        ?patient a :Patient .

        ?patient :hasRecommendedActivity ?recommendedActivity .
        MINUS {OPTIONAL {?patient :hasPhysicalActivityHabit ?recommendedActivity .}}

        ?day a :Day .
        ?day :dayDate ?date .

        MINUS {OPTIONAL {?day a :BadWeather .
                         ?day :unsuitableActivity ?recommendedActivity .}}
      }
      """

    if name != None:
        n = name.replace("userKG:", "")
        uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
        res = graph.query(sparql, initBindings={'patient': uri,
                                                'date': Literal(date, datatype=XSD.date)})
    else:
        res = graph.query(sparql, initBindings={'date': Literal(date, datatype=XSD.date)})

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager)) for c1, c2 in res]
    df = pd.DataFrame(res, columns=["Patient", "RcommendedActivity"])
    return df

In [102]:
recommended_activities_weather(userKG, "2024-03-21").head()

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_balance
1,userKG:John,userKG:activity_bodyweight
2,userKG:John,userKG:activity_dancing
3,userKG:John,userKG:activity_pilates
4,userKG:John,userKG:activity_stretching


In [103]:
recommended_activities_weather(userKG, "2024-03-21", "userKG:John")

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_balance
1,userKG:John,userKG:activity_bodyweight
2,userKG:John,userKG:activity_dancing
3,userKG:John,userKG:activity_pilates
4,userKG:John,userKG:activity_stretching
5,userKG:John,userKG:activity_swimming
6,userKG:John,userKG:activity_weightlifting
7,userKG:John,userKG:activity_yoga


# **4. Recommended Diet**

### 4.1. Recommended diets for all patients or for a specific patient

In [104]:
def recommended_diet(graph, name=None):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?recommendedDiet WHERE {
        ?patient a :Patient .

        ?patient :hasRecommendedDiet ?recommendedDiet .

        MINUS {OPTIONAL {?patient :hasDietPreference ?recommendedDiet .}}
      }
      """

    if name != None:
        n = name.replace("userKG:", "")
        uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
        res = graph.query(sparql, initBindings={'patient': uri})
    else:
        res = graph.query(sparql)

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager)) for c1, c2 in res]
    df = pd.DataFrame(res, columns=["Patient", "RecommendedDiet"])
    return df

In [105]:
recommended_diet(userKG)

,Patient,RecommendedDiet
0,userKG:John,userKG:diet_mediterranian
1,userKG:John,userKG:diet_pescatarian
2,userKG:John,userKG:diet_vegan
3,userKG:John,userKG:diet_vegetarian
4,userKG:Patient1,userKG:diet_pescatarian
...,...,...
226,userKG:Patient98,userKG:diet_vegan
227,userKG:Patient98,userKG:diet_vegetarian
228,userKG:Patient99,userKG:diet_mediterranian
229,userKG:Patient99,userKG:diet_pescatarian


In [106]:
recommended_diet(userKG, "userKG:John")

,Patient,RecommendedDiet
0,userKG:John,userKG:diet_mediterranian
1,userKG:John,userKG:diet_pescatarian
2,userKG:John,userKG:diet_vegan
3,userKG:John,userKG:diet_vegetarian


# **5. Individualized recommended activities: sorted according to the values prioritization, considering financial and time constraints, injuries and weather for a specific patient**

In [107]:
def individualized_recommended_activities(graph, name, date="2024-03-21"):

    sparql = """
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX : <http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#>

      SELECT ?patient ?value (COUNT(?secondaryValue) AS ?nSecondaryValues)
      ?recommendedActivity WHERE {
        ?patient a :Patient .

        OPTIONAL {?patient :hasValue ?valueInd .
                  ?valueInd a ?value .
                  ?value rdfs:subClassOf :Value .}

        OPTIONAL {?valueInd :prioritizedOver ?secondaryValueInd .
                  ?secondaryValueInd a ?secondaryValue .
                  ?secondaryValue rdfs:subClassOf :Value .}

        OPTIONAL {?valueInd :relevantActivity ?recommendedActivity .}

        ?patient :hasRecommendedActivity ?recommendedActivity .
        MINUS {OPTIONAL {?patient :hasPhysicalActivityHabit ?recommendedActivity .}}

        MINUS {OPTIONAL {?patient :hasChallenge ?challenge .
                         ?recommendedActivity :incompatibleWithConstraint ?challenge .}}

        MINUS {OPTIONAL {?patient :hasInjury ?injury .
                         ?recommendedActivity :incompatibleWithInjury ?injury .}}

        ?day a :Day .
        ?day :dayDate ?date .

        MINUS {OPTIONAL {?day a :BadWeather .
                         ?day :unsuitableActivity ?recommendedActivity .}}

        FILTER(?value != :Value)
        FILTER(!bound(?secondaryValue) || ?secondaryValue != :Value)
      }
      GROUP BY ?patient ?value ?nSecondaryValues ?recommendedActivity
      ORDER BY DESC(?nSecondaryValues)
      """

    n = name.replace("userKG:", "")
    uri = URIRef("http://www.semanticweb.org/aledpro/ontologies/2024/2/userKG#" + n)
    res = graph.query(sparql, initBindings={'patient': uri,
                                            'date': Literal(date, datatype=XSD.date)})

    res = list(res)
    res = [(c1.n3(graph.namespace_manager), c2.n3(graph.namespace_manager),
            c3.n3(graph.namespace_manager), c4.n3(graph.namespace_manager)) for c1, c2, c3, c4 in res]
    df = pd.DataFrame(res, columns=["Patient", "Value", "NumberOfSecondaryValues",
                                    "RcommendedActivity"])
    return df.drop_duplicates("RcommendedActivity").drop(["NumberOfSecondaryValues",
                                                          "Value"],
                                                         axis=1)

In [108]:
individualized_recommended_activities(userKG, "userKG:John")

,Patient,RcommendedActivity
0,userKG:John,userKG:activity_pilates
1,userKG:John,userKG:activity_yoga


In [113]:
individualized_recommended_activities(userKG, "userKG:Patient3")

,Patient,RcommendedActivity
0,userKG:Patient3,userKG:activity_yoga
1,userKG:Patient3,userKG:activity_pilates
2,userKG:Patient3,userKG:activity_stretching
